In [22]:
import os
def cal_result(exp_name, criterion='best', verbose=False, draw_cm=False):
    import dill as pickle
    data_stream = 70
    week_n = 7000 
    cnt = 0
    tp = 0
    fp = 0
    sum_a = 0
    sum_n = 0
    AVG = {'tpr':0.0,'fpr':0.0, 'AUC' :0.0} 

    with open('data/y.pkl', 'rb') as f:
        y = pickle.load(f).flatten().tolist()

    for stream in range(36, data_stream + 1): 
        pre = (stream - 1) * 7000
        
        flag = False 
        start, end = pre + week_n, pre + 2 * week_n
        for j in range(start, end): 
            if y[j] == 1: 
                flag = True; sum_a += 1 
            else: sum_n += 1

        def read_score(exp_name, week): 
            lst = []
            exp_name = os.path.join(os.path.join("exp3", exp_name), exp_name)
            with open(exp_name + "Week" + str(week) +  "_result.txt", "r") as f: 
                sc = float(f.readline()) 
                while sc != None: 
                    lst.append(sc)

                    s = f.readline()
                    # while !s: s = f.readline()
                    if s == '': break  
                    sc = float(s) 

            return lst                

        if flag : 
            cnt += 1
            x_score = read_score(exp_name, stream)
            from sklearn.metrics import roc_curve, auc
            GT = y[start: end]
            from sklearn.metrics import roc_auc_score
            AUC = roc_auc_score(GT, x_score)
            if verbose:  print(stream, " week :", AUC)  #*
            
            # print(len(x_score), GT) 
            FPR, TPR, thresholds = roc_curve(GT, x_score)
            maxindex = (TPR-FPR).tolist().index(max(TPR-FPR))
            threshold = thresholds[maxindex]
            tpr, fpr = TPR[maxindex], FPR[maxindex]
            if verbose: print("tpr: %f , fpr: %f, thresh: %f\n" % (tpr, fpr, threshold)) #*

            tp += sum([1 if x >= threshold and y == 1 else 0 for x, y in zip(x_score, GT)])
            fp += sum([1 if x >= threshold and y == 0 else 0 for x, y in zip(x_score, GT)])
            for i in AVG.keys(): 
                AVG[i] += eval(i)
            # if criterion == 'bset': 
                # JD = [x - y for x, y in zip(TPR, FPR)]
    # for i in AVG.keys():
    #      AVG[i] /= cnt
    print(tp, sum_a, fp, sum_n)
    fn = sum_a - tp; tn = sum_n - fp 
    if draw_cm: 
        import matplotlib.pyplot as plt
        import numpy as np
        from sklearn.metrics import confusion_matrix
        from sklearn.metrics import ConfusionMatrixDisplay
        cm = np.array([[tn, fp], [fn, tp]])
        print(cm)
        ig = plt.figure()
        plt.matshow(cm)
        plt.title('Confusion Matrix')
        plt.colorbar()
        plt.ylabel('True Label')
        plt.xlabel('Predicated Label')
        for first_index in range(len(cm)):
            for second_index in range(len(cm[first_index])):
                plt.text(first_index, second_index, cm.T[first_index][second_index])
        plt.savefig('confusion_matrix' + exp_name +'.jpg')

        plt.close('all')
    return {'tpr':tp / sum_a,'fpr': fp / sum_n, "TP":tp , "FN":fn, "FP": fp, "TN": tn}

# cal_result("exp_icarl_NN_0")


In [23]:
def f(p):
    import pandas as pd 
    flag = False
    df = None
    # p = ["exp_naive_"]

    for i in range(0, 5): 
        tmp = cal_result(p[0] + "", verbose=False,draw_cm=False) # draw_cm: 是否可视化  #! to be relay
        if flag != False: df = pd.concat([df, pd.DataFrame(data=tmp, index=[0])], axis=0) 
        else : df = pd.DataFrame(data=tmp, index=[0]); flag = True
    return df

# display(df)
# df.describe()
# pd.DataFrame(data=df.describe().loc['mean', :].to_dict(), index=[0])

In [28]:
# idx = [('FineTune', 'exp_finetune_' ),("icarl","exp_icarl_"),("icarl with NN","exp_icarl_NN_"),('随机初始化AE',"exp_randominit_"),("随机输出分数", "exp_randomoutput_"), ("非0特征数作为分数", "exp_naive_")]
idx = [('LwF', 'exp_test' ),('LwF2', 'exp_tes_0t' )]

In [29]:
y = [f([idx[i][1]]) for i in range(0, 2)]
import pandas as pd 
tmp = [pd.DataFrame(data=x.describe().loc['mean', :].to_dict(), index=[0]) for x in y ]
d = pd.concat(tmp, axis=0)

480 583 55251 244417
480 583 55251 244417
480 583 55251 244417
480 583 55251 244417
480 583 55251 244417
478 583 59541 244417
478 583 59541 244417
478 583 59541 244417
478 583 59541 244417
478 583 59541 244417


In [30]:
d.index = list(list(zip(*idx))[0])

# p = d.style.set_properties(**{'text-align': 'left'})
d

,tpr,fpr,TP,FN,FP,TN
LwF,0.823328,0.226052,480.0,103.0,55251.0,189166.0
LwF2,0.819897,0.243604,478.0,105.0,59541.0,184876.0


In [ ]:
import pandas as pd 
flag = False
df = None
p = ["exp_finetune_0", "exp_icarl_0", "exp_icarl_NN_0"]


for i in p: 
    j = i
    tmp = cal_result(j, verbose=False) 
    print(tmp)
    if flag != False: df = pd.concat([df, pd.DataFrame(data=tmp, index=[0])], axis=0) 
    else : df = pd.DataFrame(data=tmp, index=[0]); flag = True

df.describe().loc['mean', :].to_dict()


512 583 58154 244417
{'tpr': 0.8782161234991424, 'fpr': 0.23792944025988372, 'TP': 512, 'FN': 71, 'FP': 58154, 'TN': 186263}
490 583 57064 244417
{'tpr': 0.8404802744425386, 'fpr': 0.2334698486602814, 'TP': 490, 'FN': 93, 'FP': 57064, 'TN': 187353}
547 583 32059 244417
{'tpr': 0.9382504288164666, 'fpr': 0.1311651808180282, 'TP': 547, 'FN': 36, 'FP': 32059, 'TN': 212358}


{'tpr': 0.885648942252716,
 'fpr': 0.20085482324606443,
 'TP': 516.3333333333334,
 'FN': 66.66666666666667,
 'FP': 49092.333333333336,
 'TN': 195324.66666666666}